In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

## Extreme Gradient Boosting (XGBoost) Model:

#### Similar to our choice of using an XGBoost model for this task was primarily to experiment with a more robust model than a traditional linear regression model.  We did initially experiment with a Dense Neural Network (DNN) for linear regression, however, our performance (MAE ~ 2.5) was poorer than our performance using an XGBoost model - as seen in our final performance metrics below.

In [ ]:
df = pd.read_csv('transformed_df.csv')
df.drop(['Question', 'Unnamed: 24'], axis=1, inplace=True)
df.head()

,Number of Sources,Trust Score,chatGPT,copilot,gemini,llama,perplexity,Economics & Commerce,Entertainment,Food & Cooking,...,y20india.in,yahoo.com,yale.edu,yaleclimateconnections.org,yellowbrick.co,yourdictionary.com,yourstory.com,youtube.com,zavvi.com,zerohanger.com
0,2,10,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,9,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,11,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,3,11,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,9,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
X = df.drop('Trust Score', axis=1)
y = df['Trust Score']


kf_2 = KFold(n_splits =5, shuffle=True, random_state=42)
mae_scores = []

for train_index, val_index in kf_2.split(X):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)

    dtrain = xgb.DMatrix(X_train, label=y_train)
    dval = xgb.DMatrix(X_val, label=y_val)

    params = {
        'objective': 'reg:squarederror',  
        'learning_rate': 0.05,
        'max_depth': 6,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'random_state': 42,
    }

    model = xgb.train(
        params,
        dtrain,
        num_boost_round=500,
        evals=[(dval, 'validation')],
        early_stopping_rounds=10,
        verbose_eval=False,
    )

    y_pred = model.predict(dval)

    mae = mean_absolute_error(y_val, y_pred)
    mae_scores.append(mae)
    print(f"Fold MAE: {mae:.2f}")

mean_mae = np.mean(mae_scores)
print(f"Mean MAE across folds: {mean_mae:.2f}")

Fold MAE: 0.28
Fold MAE: 0.27
Fold MAE: 0.28
Fold MAE: 0.32
Fold MAE: 0.28
Mean MAE across folds: 0.29


## Comments:

#### As can be seen above, our model uses a K-Fold approach to test various different training/test sets.  All of our folds performed relatively well (highest MAE was 0.32), especially compared to the DNN's ~2.5 MAE that we initially explored.  Of course, we would like to see our MAE get closer to 0, so there is still room for improvement, which will be our focus in the future.  Ultimatley, our goal of having a model predicting trust scores is to eventually have a tool for model developers to use to do an assessment of their model's 'credibility' and transparency.